In [1]:
import requests
import pandas as pd


In [2]:
base_url = "https://api.enerpricedata.com"   # Replace with your FastAPI base URL
api_key = "sk-dUtFyE9MpUJOLuRWeBxRkvQfA__96Kb548fvptJg9-4"           # Replace with your actual API key


In [4]:
headers = {
    "X-API-Key": api_key  # FastAPI expects the key in this custom header
}


In [5]:
response = requests.get(f"{base_url}/openapi.json")
for path in response.json().get("paths", {}):
    print(path)


/graph/api/v1/energy-futures/graph-data
/graph/api/v1/energy-futures/load-zones
/graph/api/v1/rec/graph-data
/graph/api/v1/rec/states
/graph/api/v1/utility-price/graph-data
/graph/api/v1/utility-price/states
/graph/api/v1/utility-price/utilities
/graph/api/v1/utility-price/rate-classes
/graph/api/v1/ancillary-uplift/graph-data
/graph/api/v1/ancillary-uplift/load-zones
/graph/api/v1/ancillary-uplift/capacity-zones
/users
/users/{user_id}
/login
/verify-otp
/me
/logout
/forgot-password/request
/forgot-password/verify
/change-password
/datasets/upload/utility-price
/datasets/download/utility-price
/datasets/download/utility-price/summary/csv
/datasets/download/utility-price/details/csv
/datasets/download/utility-price/csv
/datasets/download/utility-price/json
/datasets
/datasets/{dataset_id}
/datasets/{dataset_id}/utility-price/summary
/datasets/{dataset_id}/utility-price/details
/datasets/{dataset_id}/utility-price/details/{detail_id}/ptc-values
/datasets/upload/energy-futures
/datasets/

Default:

In [18]:
import requests

base_url = "https://api.enerpricedata.com"
endpoint = "/me"

response = requests.get(f"{base_url}{endpoint}", headers=headers)

if response.status_code == 200:
    print("User Info:", response.json())
else:
    print("Error:", response.status_code, response.text)


User Info: {'id': 'f78fdfbf-3b5a-4e9f-8b8e-b2595da404ca', 'name': 'santosh', 'email': 'santosh@azr-holdings.com', 'role': 'admin', 'is_active': True, 'created_at': '2025-08-10T06:41:16.748649', 'updated_at': '2025-08-24T20:18:49.083073', 'api_key_id': 'b426811d-59c0-47e6-98f6-a1b2b82fdfff'}


Energy Futures:

In [5]:
import requests

# Download Energy Futures Data
url = "https://api.enerpricedata.com/datasets/download/energy-futures"
headers = {
    "X-API-Key": api_key
}
params = {
    "start_operating_date": "2025-09-02",
    "control_area": "ERCOT",
    "block_types": "7x8,2x16"
}

response = requests.get(url, headers=headers, params=params)

control_area = params["control_area"]
end_op_date = params.get("end_operating_date", params["start_operating_date"])
filename = f"EPD_EnergyFutures_{control_area}_{end_op_date}.xlsx"


if response.status_code == 200:
    with open(filename, "wb") as f:
        f.write(response.content)
    print(f"File downloaded successfully and saved as '{filename}'")
else:
    # If an error occurs, print status and response text
    print(f"Error {response.status_code}: {response.text}")
    
print("Energy futures data downloaded successfully!")

File downloaded successfully and saved as 'EPD_EnergyFutures_ERCOT_2025-09-02.xlsx'
Energy futures data downloaded successfully!


EXCEL

In [ ]:
endpoint = "/datasets/download/energy-futures"

params = {
    "start_operating_date": "2025-08-25",        # Required
    "end_operating_date": "2025-08-25",          # Optional, for date range
    "control_area": "ERCOT",                     # ERCOT, PJM, ISONE, etc.
    "block_types": "",                           # Optional: e.g., 7x8,2x16,5x16, etc. e.g., "7x8","2x16","5x16" -- By Default ALL 
    "start_date": "2025-08-01",                  # Optional: filter start
    "end_date": "2030-09-01"                     # Optional: filter end
}

response = requests.get(f"{base_url}{endpoint}", headers=headers, params=params)

control_area = params["control_area"]
end_op_date = params.get("end_operating_date", params["start_operating_date"])
filename = f"EPD_EnergyFutures_{control_area}_{end_op_date}.xlsx"


if response.status_code == 200:
    with open(filename, "wb") as f:
        f.write(response.content)
    print(f"File downloaded successfully and saved as '{filename}'")
else:
    # If an error occurs, print status and response text
    print(f"Error {response.status_code}: {response.text}")

✅ File downloaded successfully and saved as 'EPD_EnergyFutures_ERCOT_2025-08-25.xlsx'


CSV:

In [ ]:
endpoint = "/datasets/download/energy-futures/csv"

params = {
    "start_operating_date": "2025-08-25",        # Required
    "end_operating_date": "2025-08-25",          # Optional, for date range
    "control_area": "ERCOT",                     # ERCOT, PJM, ISONE, etc.
    "block_types": "",                           # Optional: e.g., 7x8,2x16,5x16, etc. e.g., "7x8","2x16","5x16" -- By Default ALL 
    "start_date": "2025-08-01",                  # Optional: filter start
    "end_date": "2030-09-01"                     # Optional: filter end
}

response = requests.get(f"{base_url}{endpoint}", headers=headers, params=params)

control_area = params["control_area"]
end_op_date = params.get("end_operating_date", params["start_operating_date"])
filename = f"EPD_EnergyFutures_{control_area}_{end_op_date}.csv"


if response.status_code == 200:
    with open(filename, "wb") as f:
        f.write(response.content)
    print(f"File downloaded successfully and saved as '{filename}'")
else:
    # status and response text
    print(f"Error {response.status_code}: {response.text}")

File downloaded successfully and saved as 'EPD_EnergyFutures_ERCOT_2025-08-25.csv'


JSON:

In [ ]:
import json

endpoint = "/datasets/download/energy-futures/json"

params = {
    "start_operating_date": "2025-08-25",        # Required
    "end_operating_date": "2025-08-25",          # Optional, for date range
    "control_area": "ERCOT",                     # ERCOT, PJM, ISONE, etc.
    "block_types": "",                           # Optional: e.g., 7x8,2x16,5x16, etc. e.g., "7x8","2x16","5x16" -- By Default ALL 
    "start_date": "2025-08-01",                  # Optional: filter start
    "end_date": "2030-09-01"                     # Optional: filter end
}

response = requests.get(f"{base_url}{endpoint}", headers=headers, params=params)

control_area = params["control_area"]
end_op_date = params.get("end_operating_date", params["start_operating_date"])
filename = f"EPD_EnergyFutures_{control_area}_{end_op_date}.json"


if response.status_code == 200:
    data = response.json()
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)
    print(f"File downloaded successfully and saved as '{filename}'")
else:
    print(f"Error {response.status_code}: {response.text}")

File downloaded successfully and saved as 'EPD_EnergyFutures_ERCOT_2025-08-25.json'


Utility Price:

In [ ]:
endpoint = "/datasets/download/utility-price"

params = {
    "start_operating_date": "2025-08-15",        # Required in YYYY-MM-DD format
    "end_operating_date": "2025-08-15",          # Optional, for date range
}

response = requests.get(f"{base_url}{endpoint}", headers=headers, params=params)

end_op_date = params.get("end_operating_date", params["start_operating_date"])
filename = f"EPD_UtilityPrice_{end_op_date}.xlsx"


if response.status_code == 200:
    with open(filename, "wb") as f:
        f.write(response.content)
    print(f"File downloaded successfully and saved as '{filename}'")
else:
    # If an error occurs, print status and response text
    print(f"Error {response.status_code}: {response.text}")

File downloaded successfully and saved as 'EPD_UtilityPrice_2025-08-15.xlsx'


CSV:

In [17]:
endpoint = "/datasets/download/utility-price/csv"

params = {
    "start_operating_date": "2025-08-15",        # Required in YYYY-MM-DD format
    #"end_operating_date": "2025-08-15",          # Optional, for date range
}

response = requests.get(f"{base_url}{endpoint}", headers=headers, params=params)

end_op_date = params.get("end_operating_date", params["start_operating_date"])
filename = f"EPD_UtilityPrice_{end_op_date}.zip"


if response.status_code == 200:
    with open(filename, "wb") as f:
        f.write(response.content)
    print(f"File downloaded successfully and saved as '{filename}'")
else:
    #status and response text
    print(f"Error {response.status_code}: {response.text}")

File downloaded successfully and saved as 'EPD_UtilityPrice_2025-08-15.zip'


JSON: -- Clarity


In [ ]:
import json

endpoint = "/datasets/download/utility-price/json"

params = {
    "start_operating_date": "2025-08-15",        # Required
    "end_operating_date": "2025-08-15",          # Optional, for date range
}

response = requests.get(f"{base_url}{endpoint}", headers=headers, params=params)

end_op_date = params.get("end_operating_date", params["start_operating_date"])
filename = f"EPD_UtilityPrice_{end_op_date}.json"


if response.status_code == 200:
    data = response.json()
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)
    print(f"File downloaded successfully and saved as '{filename}'")
else:
    print(f"Error {response.status_code}: {response.text}")